In [1]:
# Imports

import numpy as np
import pandas as pd
import pandas_profiling as pp


In [2]:
# Step 1: Read in the tables. Without encoding = "ISO-8859-1" for users, receive unicodeDecodeError of invalid continuation byte

engagement = pd.read_csv('takehome_user_engagement.csv')
users = pd.read_csv('takehome_users.csv', encoding = "ISO-8859-1")


In [3]:
# Step 2: Explore data using pandas profiling. Do these in two separate notebook cells.

pp.ProfileReport(engagement)

In [ ]:
pp.ProfileReport(users)

<h4>The challenge asks us to get users who are active 3 or more days in a 7-day period. This list will get users if they were active 3 times in a week, with weeks starting on Sunday. It does not catch "rolling" users who might have logged in only on Saturday, Sunday, and Monday of the next week, for example, and also counts users that may have visited three times in one day as an active user.</h4>

In [ ]:
# Change timestamp to pd.datetime from categorical
engagement['time_stamp'] = pd.to_datetime(engagement['time_stamp'])

In [ ]:
# Group by both engagement and timestamp to see if user visited 3 or more times in a week.
df = engagement.groupby(['user_id', engagement['time_stamp'].dt.week]).sum() >= 3

# Get user_ids by calling groupby on 'user_id'. Get the indices as a list by calling index and changing it tolist.
active_users = df[df['visited'] == True].groupby('user_id').sum().index.tolist()

In [ ]:
# Add a true/false column to users if user is active

users['active_user'] = users.object_id.isin(active_users)

<h4> Data cleaning and feature engineering: </h4>
<ul>
1.) Set the index as object_id. <br>
2.) Change creation_time to datetime object. Get year, month, day of week for features. <br>
3.) Check if there are common (n > 10) last names. <br>
4.) Create email_domain column with email domain (regexed) <br>
5.) Label encode creation_source will be done in sklearn <br>
6.) Last-session-created is going to be dropped. Too related to our target variable. <br>
7.) Opted in mailing list is fine <br>
8.) Enabled for drip is fine<br>
9.) Gets whether the user who referred the user is still active.<br>
</ul>

In [ ]:
# Step 1.) Set index as object_id

users = users.set_index('object_id')

In [ ]:
# 2.) Change creation_time to datetime object. Get year, month, day of week for features.

users['creation_time'] = pd.to_datetime(users['creation_time'])
users['year'] = users['creation_time'].dt.year
users['month'] = users['creation_time'].dt.month
users['day_of_week'] = users['creation_time'].dt.day_name()

users.head()

In [ ]:
# 3.) Check to see if common last name. Use common last names as features to check relationships.

users['last_name'] = users['name'].apply(lambda x: x.split(" ")[0])

In [ ]:
# This is a list of the 50 most common last names. Interesting to check to see if certain last names are more active.


users.last_name.value_counts()
top50 = users.last_name.value_counts().nlargest(50).index
users['last_name_common'] = users['last_name'].where(users['last_name'].isin(top50), other='Other')

In [ ]:
# 4.) Create email_domain column with email domain (regexed)

users['email_domain'] = users['email'].apply(lambda x: x.split('@')[-1])

In [ ]:
# Reduces the number of common email domains to six. (Lots of domains with 2-3 users each.)
topsix = users.email_domain.value_counts().nlargest(6).index

users['email_updated'] = users['email_domain'].where(users['email_domain'].isin(topsix), other='Other')
users['email_updated'].value_counts()

# See https://www.dataschool.io/python-pandas-tips-and-tricks/ #77

In [ ]:
users.head()

In [ ]:
# 6.) Last-session-created is going to be dropped. Too related to our target variable.
# 7.) Opted in mailing list is fine
# 8.) Enabled for drip is fine

In [ ]:
# 9.) Change invited_by_user_id into string and check if that string is an active user.

users['invited_by_user_id'].fillna(0, inplace=True)
users['invited_by_user_id'] = users['invited_by_user_id'].apply(lambda x: str(int(x)))

users.head()

In [ ]:
# 9.) Gets whether the user who referred the user is still active. Common thing to watch out for in retention modeling.

users['invited_by_active_user'] = users['invited_by_user_id'].apply(lambda x: int(x) in active_users)

In [ ]:
users['invited'] = users['invited_by_user_id'].apply(lambda x: int(x) > 0)

In [ ]:
# Create X and y matrices for machine learning.

X = users[['creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'year',
                      'month', 'day_of_week', 'last_name_common', 'email_updated', 'invited_by_active_user']]
y = users['active_user']

<h3>Machine Learning and Statistical Modeling</h3>

In [ ]:
# Build a pipeline to dummy encode categorical features
from sklearn.preprocessing import OneHotEncoder

# use when different features need different preprocessing
from sklearn.compose import make_column_transformer

ohe = OneHotEncoder(sparse=False)

X = X.astype(str)

In [ ]:
# One hot encode all columns

column_trans = make_column_transformer(
    (OneHotEncoder(), ['creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'year',
                      'month', 'day_of_week', 'last_name_common', 'email_updated', 'invited_by_active_user']),
    remainder='drop')

In [ ]:
# Create a matrix with encoded columns. This loses column names which is a pain.

transformed = column_trans.fit_transform(X)

In [ ]:
# Apply logistic regression and random forest models. 

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [ ]:
# Create pipeline to encode columns and run predictor for each model. 
# Useful to grid search the pipeline instead of individual models. 

from sklearn.pipeline import make_pipeline
pipe_lr = make_pipeline(column_trans, logreg)
pipe_rf = make_pipeline(column_trans, rf)

In [ ]:
# Score each of the models with cross validation. Baseline is random guess based on proportion of active users.

from sklearn.model_selection import cross_val_score
print('Baseline: ', 1 - len(active_users) / len(y))
print('Logistic Regression: ', cross_val_score(pipe_lr, X, y, cv=10, scoring='accuracy').mean())
print('Random Forest: ', cross_val_score(pipe_rf, X, y, cv=10, scoring='accuracy').mean())

<h3>At this point, we have run two models, logistic regression and random forest. Neither has beaten the baseline of guessing that none of the users will be active. Things to try from here:</h3>
    <ul>
    <li>Other ML models (XGBooxt)<br></li>
    <li>Variable selection processes to prevent overfitting<br></li>
    <li>Improvement on figuring out who is an "active user"<br></li>
    <li>Messing around with hyperparameter tuning</li>
   </ul>
   